# Group by in  Spark

In [3]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [4]:
df_green = spark.read.parquet('data/pq/green/*/*')

In [6]:
df_green.registerTempTable('green')

/usr/local/anaconda3/lib/python3.12/site-packages/pyspark/sql/dataframe.py:329: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [11]:
df_green_revenue = spark.sql("""
SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [12]:
df_green_revenue.show()

+-------------------+----+------------------+--------------+
|               hour|zone|            amount|number_records|
+-------------------+----+------------------+--------------+
|2020-01-24 09:00:00|  81|             59.49|             2|
|2020-01-04 21:00:00|  25|            513.83|            32|
|2020-01-10 19:00:00|  66| 545.6800000000001|            27|
|2020-01-30 07:00:00|  75| 556.6600000000001|            40|
|2020-01-18 01:00:00| 260|            144.56|            12|
|2020-01-12 08:00:00| 177|31.090000000000003|             2|
|2020-01-20 21:00:00| 166|            133.28|            12|
|2020-01-03 04:00:00|  14|            105.34|             2|
|2020-01-30 20:00:00|  74| 766.4400000000002|            58|
|2020-01-02 16:00:00|  66|             257.0|            12|
|2020-01-11 23:00:00|  42|340.46999999999997|            31|
|2020-01-13 12:00:00|  89|            135.25|             6|
|2020-01-02 08:00:00| 174|50.309999999999995|             3|
|2020-01-20 01:00:00|  4

In [13]:
# write the results to parquet
df_green_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/green', mode='overwrite')

25/02/24 15:13:29 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 95,00% for 8 writers
25/02/24 15:13:29 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 84,44% for 9 writers
25/02/24 15:13:29 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 76,00% for 10 writers
25/02/24 15:13:29 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 69,09% for 11 writers
25/02/24 15:13:29 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 63,33% for 12 writers
25/02/24 15:13:29 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 58,46% for 13 writers
25/02/24 15:13:29 WARN MemoryManager: Total allocation exceeds 95,

In [14]:
df_yellow = spark.read.parquet('data/pq/yellow/*/*')
df_yellow.registerTempTable('yellow')

/usr/local/anaconda3/lib/python3.12/site-packages/pyspark/sql/dataframe.py:329: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [15]:
df_yellow_revenue = spark.sql("""
SELECT 
    date_trunc('hour', tpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    yellow
WHERE
    tpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [16]:
df_yellow_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/yellow', mode='overwrite')

25/02/24 15:18:27 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 95,00% for 8 writers
25/02/24 15:18:27 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 84,44% for 9 writers
25/02/24 15:18:27 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 76,00% for 10 writers
25/02/24 15:18:27 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 69,09% for 11 writers
25/02/24 15:18:27 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 63,33% for 12 writers
25/02/24 15:18:27 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 58,46% for 13 writers
25/02/24 15:18:27 WARN MemoryManager: Total allocation exceeds 95,

In [17]:
df_green_revenue = spark.read.parquet('data/report/revenue/green')
df_yellow_revenue = spark.read.parquet('data/report/revenue/yellow')

We will join the tables to produce the following result:

<img src="./img/join_spark.png" width="80%">

In [18]:
df_green_revenue_tmp = df_green_revenue \
    .withColumnRenamed('amount', 'green_amount') \
    .withColumnRenamed('number_records', 'green_number_records')

df_yellow_revenue_tmp = df_yellow_revenue \
    .withColumnRenamed('amount', 'yellow_amount') \
    .withColumnRenamed('number_records', 'yellow_number_records')

In [19]:
# we will join the two dataframes on the hour and zone columns
# we will use an outer join to keep all the records

df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=['hour', 'zone'], how='outer')

In [20]:
df_join.write.parquet('data/report/revenue/total', mode='overwrite') # write the results to parquet

25/02/24 15:27:11 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 95,00% for 8 writers
25/02/24 15:27:11 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 84,44% for 9 writers
25/02/24 15:27:11 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 76,00% for 10 writers
25/02/24 15:27:11 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 69,09% for 11 writers
25/02/24 15:27:11 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 63,33% for 12 writers
25/02/24 15:27:11 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 58,46% for 13 writers
25/02/24 15:27:11 WARN MemoryManager: Total allocation exceeds 95,

In [21]:
df_join_read = spark.read.parquet('data/report/revenue/total')

In [ ]:
df_join_read.show() #when we call show, we will execute all the steps up to here

+-------------------+----+------------------+--------------------+------------------+---------------------+
|               hour|zone|      green_amount|green_number_records|     yellow_amount|yellow_number_records|
+-------------------+----+------------------+--------------------+------------------+---------------------+
|2020-01-01 00:00:00|   4|              NULL|                NULL|1004.3000000000002|                   57|
|2020-01-01 00:00:00|  50|              NULL|                NULL| 4177.479999999998|                  183|
|2020-01-01 00:00:00|  63|              51.9|                   2|              70.8|                    1|
|2020-01-01 00:00:00|  74|317.09000000000003|                  24| 586.2100000000002|                   47|
|2020-01-01 00:00:00|  82|            305.12|                  28|              29.8|                    1|
|2020-01-01 00:00:00| 116|328.76000000000005|                  18|            561.51|                   34|
|2020-01-01 00:00:00| 134| 6

In [ ]:
# example of one big df and one small one
# when you have a join with a small table, each executor gets a copy of the data
# this means
# we don't need to re-suffle the data, we can just broadcast the small table to all the executors
# this is done by calling the broadcast function on the small dataframe

# first we will read zones
df_zones = spark.read.parquet('zones/')

In [25]:
# now we will join the tables
df_result = df_join.join(df_zones, df_join.zone == df_zones.LocationID)

In [26]:
df_result.show()

+-------------------+----+------------------+--------------------+------------------+---------------------+----------+---------+--------------------+------------+
|               hour|zone|      green_amount|green_number_records|     yellow_amount|yellow_number_records|LocationID|  Borough|                Zone|service_zone|
+-------------------+----+------------------+--------------------+------------------+---------------------+----------+---------+--------------------+------------+
|2020-01-01 00:00:00|  55|            129.29|                   4|              NULL|                 NULL|        55| Brooklyn|        Coney Island|   Boro Zone|
|2020-01-01 00:00:00|  61|            526.71|                  17|            146.64|                    3|        61| Brooklyn| Crown Heights North|   Boro Zone|
|2020-01-01 00:00:00|  65|            199.49|                  10|            409.35|                   19|        65| Brooklyn|Downtown Brooklyn...|   Boro Zone|
|2020-01-01 00:00:00| 

In [27]:
df_result.drop('LocationID', 'zone').write.parquet('tmp/revenue-zones') # temporary save

25/02/24 15:39:07 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 95,00% for 8 writers
25/02/24 15:39:07 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 84,44% for 9 writers
25/02/24 15:39:07 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 76,00% for 10 writers
25/02/24 15:39:07 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 69,09% for 11 writers
25/02/24 15:39:07 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 63,33% for 12 writers
25/02/24 15:39:07 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 58,46% for 13 writers
25/02/24 15:39:07 WARN MemoryManager: Total allocation exceeds 95,